## Import Libraries

In [1]:
!pip install --upgrade huggingface_hub
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
import pandas as pd
from sklearn.metrics import confusion_matrix

from datasets import load_dataset, Dataset

## Load Dataset

In [3]:
### START OF CODE FROM EXTERNAL SOURCE (URL: https://huggingface.co/datasets/florentgbelidji/car-reviews)
ds = load_dataset("florentgbelidji/car-reviews")
### END OF CODE FROM EXTERNAL SOURCE (URL: https://huggingface.co/datasets/florentgbelidji/car-reviews)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train_car.csv:   0%|          | 0.00/24.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36984 [00:00<?, ? examples/s]

## EDA

In [4]:
ds['train'][10]

{'Unnamed: 0.1': 10,
 'Unnamed: 0': 10,
 'Review_Date': ' on 05/23/18 06:57 AM (PDT)',
 'Author_Name': 'Bill From MI ',
 'Vehicle_Title': '2007 Toyota Avalon Sedan Touring 4dr Sedan (3.5L 6cyl 5A)',
 'Review_Title': '280K MIles and Very Relieable',
 'Review': ' I have owned a 2007 Touring model for 3 years.  I purchased with 180K miles after my Lexus LS430 was totaled.  I looked at a number of Avalons but did not want to spend over $10K for a lower mileage one because I drive over 30K miles per year.  I bought this specific car thinking that I might drive it for a year while I looked for another LS430.  I love the handling and comfort of this car. It does not ride as soft as the LS but it easier to drive because it just soaks up the miles without much steering correction.  I tried the Limited model first but found the cooled seats were not as comfortable as those without this feature.  I specifically looked for a model that did NOT have the sunroof as this feature limits headroom (I\'m

In [5]:
reviews = ds['train']['Review']

In [6]:
reviews[10]

' I have owned a 2007 Touring model for 3 years.  I purchased with 180K miles after my Lexus LS430 was totaled.  I looked at a number of Avalons but did not want to spend over $10K for a lower mileage one because I drive over 30K miles per year.  I bought this specific car thinking that I might drive it for a year while I looked for another LS430.  I love the handling and comfort of this car. It does not ride as soft as the LS but it easier to drive because it just soaks up the miles without much steering correction.  I tried the Limited model first but found the cooled seats were not as comfortable as those without this feature.  I specifically looked for a model that did NOT have the sunroof as this feature limits headroom (I\'m 6\'2") and need the extra 3" or so that models without the sunroom have.  I have put a new battery and brakes on this car.  Those have been the only repairs in 100K miles.  No check engine lights.  No weird noises.  Great acceleration.  This is a roomy car wi

## Model 1 - Transformers

### Balanced Sampling

In [7]:
df = pd.DataFrame(ds['train'])

In [8]:
min_samples = df['Rating'].value_counts().min()
print(min_samples)

742


In [9]:
balanced_df = df.groupby('Rating').apply(lambda x: x.sample(min_samples))

<ipython-input-9-1dced5f22f9e>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = df.groupby('Rating').apply(lambda x: x.sample(min_samples)) ##.reset_index(drop=True)


In [10]:
sampled_dataset = Dataset.from_pandas(balanced_df)

### Filtering Negative Feedbacks

#### Sentiment Analysis Model 1

In [11]:
### START OF CODE FROM EXTERNAL SOURCE (URL:https://huggingface.co/learn/nlp-course/chapter1/3?fw=pt)
sentiment_classifier = pipeline("sentiment-analysis", device=0)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
SA = sentiment_classifier(sampled_dataset['Review'], truncation=True)
### END OF CODE FROM EXTERNAL SOURCE (URL:https://huggingface.co/learn/nlp-course/chapter1/3?fw=pt)

In [13]:
SA

[{'label': 'NEGATIVE', 'score': 0.8969122767448425},
 {'label': 'NEGATIVE', 'score': 0.9995617270469666},
 {'label': 'NEGATIVE', 'score': 0.9988203644752502},
 {'label': 'NEGATIVE', 'score': 0.9983812570571899},
 {'label': 'NEGATIVE', 'score': 0.9996172189712524},
 {'label': 'NEGATIVE', 'score': 0.9991171956062317},
 {'label': 'NEGATIVE', 'score': 0.9996631145477295},
 {'label': 'NEGATIVE', 'score': 0.9993864297866821},
 {'label': 'NEGATIVE', 'score': 0.9823570251464844},
 {'label': 'NEGATIVE', 'score': 0.999123752117157},
 {'label': 'NEGATIVE', 'score': 0.9973121881484985},
 {'label': 'NEGATIVE', 'score': 0.9991406202316284},
 {'label': 'NEGATIVE', 'score': 0.9968884587287903},
 {'label': 'NEGATIVE', 'score': 0.9996378421783447},
 {'label': 'NEGATIVE', 'score': 0.9954022169113159},
 {'label': 'NEGATIVE', 'score': 0.9986212253570557},
 {'label': 'NEGATIVE', 'score': 0.9996114373207092},
 {'label': 'NEGATIVE', 'score': 0.9998102784156799},
 {'label': 'NEGATIVE', 'score': 0.9130446910858

In [14]:
negative_reviews = []
for review, result in zip(sampled_dataset['Review'], SA):
    if result['label'] == 'NEGATIVE' and result['score'] >= 0.8:
        negative_reviews.append(review)

In [15]:
negative_reviews

[' In less than 10 000 miles I have to replace the CVT Transmission 4 times  and still the problem. Fuel economy is like any other full side car, I will go with TOYOTA.',
 " Car has no power even with new engine. Camry is too heavy and needs 3.0L V6 engine! Transmission is not performing (because Camry is too heavy for it?). I am not impressed with gas mileage as well. With my 2009 Camry I used to get 23 mixed and 30 on Hwy (better than the sticker). Now gas mileage is below the sticker. Toyota didn't do good job on the engine-transmission side. Design is weak: looks like old Lexus ES (I understand a lot of us kinda proud of this fact, but you could get an old Lexus ES for 1/2 of Camry's price but with 3.0L engine). Interior: Please move clock to the instrument cluster! Who designed it?",
 " I have the 2014 xDrive28i model.  The seats are cheap and not comfortable for a long drive.  The acceleration is more like a Ford Escort than a BMW.  The sales guy said it is because it is a 4 cyli

#### Sentiment Analysis Model 2

In [21]:
### START OF CODE FROM EXTERNAL SOURCE (URL:https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest)
sentiment_classifier_2 = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest", batch_size=4, device=0)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
SA2 = sentiment_classifier_2(sampled_dataset['Review'], truncation=True, max_length=512)
### END OF CODE FROM EXTERNAL SOURCE (URL:https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest)

In [23]:
SA2

[{'label': 'negative', 'score': 0.8256943821907043},
 {'label': 'negative', 'score': 0.857836127281189},
 {'label': 'negative', 'score': 0.8243849873542786},
 {'label': 'negative', 'score': 0.8385199308395386},
 {'label': 'negative', 'score': 0.841106116771698},
 {'label': 'negative', 'score': 0.8328753709793091},
 {'label': 'negative', 'score': 0.8348978757858276},
 {'label': 'negative', 'score': 0.910218358039856},
 {'label': 'negative', 'score': 0.9166533946990967},
 {'label': 'negative', 'score': 0.7653681039810181},
 {'label': 'negative', 'score': 0.4526568651199341},
 {'label': 'negative', 'score': 0.8522607088088989},
 {'label': 'negative', 'score': 0.883399248123169},
 {'label': 'negative', 'score': 0.752070426940918},
 {'label': 'positive', 'score': 0.6132557988166809},
 {'label': 'neutral', 'score': 0.526540219783783},
 {'label': 'negative', 'score': 0.9385873079299927},
 {'label': 'negative', 'score': 0.9055056571960449},
 {'label': 'negative', 'score': 0.823078453540802},
 

In [24]:
negative_reviews2 = []
for review, result in zip(sampled_dataset['Review'], SA2):
    if result['label'] == 'NEGATIVE' and result['score'] >= 0.8:
        negative_reviews2.append(review)

In [25]:
negative_reviews2

[]

### Zero-Shot Classification

In [32]:
### START OF CODE FROM EXTERNAL SOURCE (URL:https://huggingface.co/facebook/bart-large-mnli)
zeroshot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0, batch_size=16)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
candidate_labels=["performance", "efficiency ", "safety", "comfort", "design", "price", "service"]

In [34]:
ZC = zeroshot_classifier(negative_reviews, candidate_labels, truncation=True, multi_label=True)
### START OF CODE FROM EXTERNAL SOURCE (URL:https://huggingface.co/facebook/bart-large-mnli)

In [35]:
ZC

[{'sequence': ' In less than 10 000 miles I have to replace the CVT Transmission 4 times  and still the problem. Fuel economy is like any other full side car, I will go with TOYOTA.',
  'labels': ['service',
   'efficiency ',
   'performance',
   'price',
   'design',
   'safety',
   'comfort'],
  'scores': [0.8883132338523865,
   0.5036608576774597,
   0.12449624389410019,
   0.09942995011806488,
   0.07267536222934723,
   0.06776759773492813,
   0.06602133065462112]},
 {'sequence': " Car has no power even with new engine. Camry is too heavy and needs 3.0L V6 engine! Transmission is not performing (because Camry is too heavy for it?). I am not impressed with gas mileage as well. With my 2009 Camry I used to get 23 mixed and 30 on Hwy (better than the sticker). Now gas mileage is below the sticker. Toyota didn't do good job on the engine-transmission side. Design is weak: looks like old Lexus ES (I understand a lot of us kinda proud of this fact, but you could get an old Lexus ES for 1

In [36]:
categories1 = [result['labels'][0] for result in ZC]
scores1 = [result['scores'][0] for result in ZC]
categories2 = [result['labels'][1] for result in ZC]
scores2 = [result['scores'][1] for result in ZC]
negative_df = pd.DataFrame({
    "Text": negative_reviews,
    "Category1": categories1,
    "Score1": scores1,
    "Category2": categories2,
    "Score2": scores2
})
negative_df

,Text,Category1,Score1,Category2,Score2
0,In less than 10 000 miles I have to replace t...,service,0.888313,efficiency,0.503661
1,Car has no power even with new engine. Camry ...,design,0.990926,performance,0.806147
2,I have the 2014 xDrive28i model. The seats a...,design,0.787387,performance,0.633503
3,Bought it used at carmax with maxcare warrant...,price,0.880193,service,0.868050
4,I Switch between Hyundai 2017 Elantra and Toy...,performance,0.895215,service,0.886890
...,...,...,...,...,...
2512,"great suv , but u made two mistakes, car shou...",performance,0.742334,safety,0.729470
2513,It is no wonder the new silverado and tundra ...,performance,0.831477,comfort,0.352530
2514,"I'm very pleased with my ""new"" Tacoma \rtruck...",comfort,0.864020,service,0.831045
2515,Just bought the Altima hybrid a few days ago.,comfort,0.930921,price,0.920989


In [37]:
grouped1 = negative_df.groupby('Category1')
result1 = grouped1.agg(Count=('Category1', 'size'), Average=('Score1', 'mean'))
result1

,Count,Average
Category1,,
comfort,188,0.814473
design,225,0.878177
efficiency,29,0.675648
performance,648,0.765665
price,420,0.805527
safety,405,0.826813
service,602,0.814446


In [38]:
grouped2 = negative_df.groupby('Category2')
result2 = grouped2.agg(Count=('Category2', 'size'), Average=('Score2', 'mean'))
result2

,Count,Average
Category2,,
comfort,267,0.677170
design,237,0.673170
efficiency,41,0.587763
performance,715,0.667923
price,410,0.669918
safety,444,0.652633
service,403,0.686161


## Evaluation

### Evaluation - Sentiment Analysis Model 1

In [16]:
scores = [item['score'] for item in SA]
print(max(scores),min(scores))

0.9998884201049805 0.5041530132293701


In [17]:
predictions = []
for result in SA:
    score = result['score']
    label = result['label']
    if label == 'NEGATIVE':
        rating = round(5 - score * 4)
    else:
        rating = round(score * 4 + 1)
    predictions.append(rating)
label = balanced_df['Rating']

In [18]:
matrix = confusion_matrix(label, predictions)
matrix

array([[704,  10,   3,   8,  17],
       [704,  17,   2,   5,  14],
       [655,  17,  12,   9,  49],
       [306,  21,  21,  24, 370],
       [110,  21,  21,  23, 567]])

In [19]:
SA_label = [result['label'] for result in SA]
SA_score = [result['score'] for result in SA]
SA_df = pd.DataFrame({
    "Text": balanced_df['Review'],
    "Label": SA_label,
    "Score": SA_score,
})
SA_df

Text     Label  \
Rating                                                                      
1      36838   In less than 10 000 miles I have to replace t...  NEGATIVE   
       14692   Car has no power even with new engine. Camry ...  NEGATIVE   
       17970   I have the 2014 xDrive28i model.  The seats a...  NEGATIVE   
       20652   Bought it used at carmax with maxcare warrant...  NEGATIVE   
       11621   I Switch between Hyundai 2017 Elantra and Toy...  NEGATIVE   
...                                                         ...       ...   
5      36620   Let me start by saying I do really like this ...  POSITIVE   
       13365   Study Consumer Reports testing of the several...  POSITIVE   
       9181    Have had my Prius V for a month and have neve...  POSITIVE   
       1817    I've done every type of wheeling that you can...  NEGATIVE   
       14998   I love my Matrix. It is the most reliable sta...  POSITIVE   

                 Score  
Rating                  
1      36838  0.896912  
       14692  0.999562  
       17970  0.998820  
       20652  0.998381  
       11621  0.999617  
...                ...  
5      36620  0.979503  
       13365  0.999723  
       9181   0.987234  
       1817   0.933916  
       14998  0.996443  

[3710 rows x 3 columns]

In [20]:
sentiment_sample = SA_df.sample(n=20, random_state=97)
sentiment_sample

Text     Label  \
Rating                                                                      
2      6797    Bought 2012 Prius two four months ago.Unless ...  NEGATIVE   
3      29790   This vehicle served it's purpose, it proveded...  NEGATIVE   
1      3994    My Sienna CE and have had an on going \rprobl...  NEGATIVE   
2      25856   I have had my Rogue for 2 weeks! It has been ...  NEGATIVE   
       10944   It's been a rather dissapointing 24 months wi...  NEGATIVE   
5      31874   Great truck for the money.  I have had two Ta...  POSITIVE   
4      12310   Wasn't the first choice for us but we like it...  POSITIVE   
5      12385   Compared to other suv s in its class it’s rea...  POSITIVE   
       31283   I purchased this vehicle the day after it was...  POSITIVE   
1      14940   I just think that it could use a lot of work,...  NEGATIVE   
2      22754   I special ordered a 2002 BMW 330 \rconvertibl...  NEGATIVE   
3      11490   2004 Corolla CE manual, now with 241k-plus. r...  NEGATIVE   
4      6312    Gas mileage is not nearly what is displayed o...  POSITIVE   
1      25620   That the CVT transmissions are junk. You can ...  NEGATIVE   
3      13009   I discovered an oil leak coming from above th...  NEGATIVE   
2      13774   I had Corolla for 9 years. I had no problem w...  NEGATIVE   
1      27793   Finally sold My Nissan Pathfinder. Brought it...  NEGATIVE   
       1975    So I buy a used Tacoma at 140,000 miles - 200...  NEGATIVE   
4      19656   They destroyed the interior and I regret my w...  NEGATIVE   
2      22798   how sad , when once you could by a premium , ...  NEGATIVE   

                 Score  
Rating                  
2      6797   0.998787  
3      29790  0.991985  
1      3994   0.999529  
2      25856  0.998915  
       10944  0.998544  
5      31874  0.998481  
4      12310  0.965613  
5      12385  0.999778  
       31283  0.975036  
1      14940  0.999783  
2      22754  0.998662  
3      11490  0.999386  
4      6312   0.999556  
1      25620  0.999699  
3      13009  0.998834  
2      13774  0.999547  
1      27793  0.997325  
       1975   0.999550  
4      19656  0.999530  
2      22798  0.999668

### Evaluation - Sentiment Analysis Model 2

In [26]:
scores2 = [item['score'] for item in SA2]
print(max(scores2),min(scores2))

0.9901902675628662 0.35100260376930237


In [27]:
predictions2 = []
for result in SA2:
    score2 = result['score']
    label2 = result['label']
    if label2 == 'NEGATIVE':
        rating2 = round(5 - score2 * 4)
    elif label2 == 'POSITIVE':
        rating2 = round(score2 * 4 + 1)
    else:
        rating2 = round((score2 - 0.3) / 0.7 * 4 + 1)
    predictions2.append(rating2)
label2 = balanced_df['Rating']

In [29]:
matrix2 = confusion_matrix(label2, predictions2)
matrix2

array([[  2, 100, 122, 382, 136],
       [  9,  92, 147, 397,  97],
       [  8, 136, 208, 330,  60],
       [  6, 129, 166, 270, 171],
       [  5,  62,  97, 189, 389]])

In [30]:
SA_label2 = [result['label'] for result in SA2]
SA_scores2 = [result['score'] for result in SA2]
SA_df2 = pd.DataFrame({
    "Text": balanced_df['Review'],
    "Label": SA_label2,
    "Score": SA_scores2,
})
SA_df2

Text     Label  \
Rating                                                                      
1      36838   In less than 10 000 miles I have to replace t...  negative   
       14692   Car has no power even with new engine. Camry ...  negative   
       17970   I have the 2014 xDrive28i model.  The seats a...  negative   
       20652   Bought it used at carmax with maxcare warrant...  negative   
       11621   I Switch between Hyundai 2017 Elantra and Toy...  negative   
...                                                         ...       ...   
5      36620   Let me start by saying I do really like this ...  positive   
       13365   Study Consumer Reports testing of the several...  positive   
       9181    Have had my Prius V for a month and have neve...  positive   
       1817    I've done every type of wheeling that you can...  positive   
       14998   I love my Matrix. It is the most reliable sta...  positive   

                 Score  
Rating                  
1      36838  0.825694  
       14692  0.857836  
       17970  0.824385  
       20652  0.838520  
       11621  0.841106  
...                ...  
5      36620  0.832047  
       13365  0.934261  
       9181   0.930375  
       1817   0.821418  
       14998  0.882844  

[3710 rows x 3 columns]

In [31]:
sentiment_sample2 = SA_df2.sample(n=20, random_state=97)
sentiment_sample2

Text     Label  \
Rating                                                                      
2      6797    Bought 2012 Prius two four months ago.Unless ...  negative   
3      29790   This vehicle served it's purpose, it proveded...   neutral   
1      3994    My Sienna CE and have had an on going \rprobl...  negative   
2      25856   I have had my Rogue for 2 weeks! It has been ...  negative   
       10944   It's been a rather dissapointing 24 months wi...  negative   
5      31874   Great truck for the money.  I have had two Ta...  positive   
4      12310   Wasn't the first choice for us but we like it...  positive   
5      12385   Compared to other suv s in its class it’s rea...  positive   
       31283   I purchased this vehicle the day after it was...  positive   
1      14940   I just think that it could use a lot of work,...  negative   
2      22754   I special ordered a 2002 BMW 330 \rconvertibl...  negative   
3      11490   2004 Corolla CE manual, now with 241k-plus. r...   neutral   
4      6312    Gas mileage is not nearly what is displayed o...  positive   
1      25620   That the CVT transmissions are junk. You can ...  negative   
3      13009   I discovered an oil leak coming from above th...  negative   
2      13774   I had Corolla for 9 years. I had no problem w...  negative   
1      27793   Finally sold My Nissan Pathfinder. Brought it...  negative   
       1975    So I buy a used Tacoma at 140,000 miles - 200...  negative   
4      19656   They destroyed the interior and I regret my w...  negative   
2      22798   how sad , when once you could by a premium , ...  negative   

                 Score  
Rating                  
2      6797   0.880154  
3      29790  0.530524  
1      3994   0.774555  
2      25856  0.745402  
       10944  0.901130  
5      31874  0.970544  
4      12310  0.839188  
5      12385  0.691183  
       31283  0.789369  
1      14940  0.927230  
2      22754  0.841077  
3      11490  0.504493  
4      6312   0.759325  
1      25620  0.748814  
3      13009  0.760703  
2      13774  0.836139  
1      27793  0.822323  
       1975   0.692218  
4      19656  0.808122  
2      22798  0.823601

### Evaluation - Zero-Shot Classification

In [39]:
negative_df_sample = negative_df.sample(n=20, random_state=97)
negative_df_sample

,Text,Category1,Score1,Category2,Score2
1915,This van is really dull first of all. Design ...,design,0.986768,service,0.526694
534,Luxurious? nopePractical? nopeEconomical? nop...,performance,0.913309,price,0.774689
358,I special ordered a 2008 BMW 528i. It was a ...,safety,0.987051,design,0.945889
554,"Stay away, the CVT transmission at Nissan is ...",safety,0.935916,performance,0.276276
475,Don't by a AWD Sienna until Toyota figures ou...,safety,0.996190,price,0.987210
481,I have had nothing but problems with \rthis c...,price,0.350834,safety,0.298459
2444,"47K miles, no problems - still runs \rlike a ...",performance,0.682953,design,0.542909
285,I purchased this vehicle approximately one ye...,service,0.974507,performance,0.579426
1245,"I purchased a 2011 Maxima paid 42,000 dollars...",service,0.859691,performance,0.557351
1223,Have only 16600 Miles.The Car already has a r...,safety,0.928742,performance,0.826249
